# Portland Vs Portland

## Introduction
The two coastal cities Portland, Oregon, and Portland, Maine, share a name, but how much more do they have in common? The population of Portland, Oregon, is approximately ten times greater than the population of Portland, Maine. They are separated by approximately 2,500 miles - 45 hours by car or ten to twelve hours by commercial flight. In this report, I will compare the two cities in three key areas: population demographics, environmental conditions, and recreational opportunities.

In [1]:
# install and import things
!pip install folium
!pip install geopy
from bs4 import BeautifulSoup 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
from pandas import json_normalize
from folium.features import DivIcon
import pandas as pd
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
import requests
import urllib
import folium
import json
pd.set_option('display.max_columns',None)

In [2]:
# get latitude and longitude for each city
OR_address = 'Portland, Oregon'
geolocator = Nominatim(user_agent="ny_explorer")
OR_location = geolocator.geocode(OR_address)
OR_latitude = OR_location.latitude
OR_longitude = OR_location.longitude

ME_address = 'Portland, Maine'
ME_location = geolocator.geocode(ME_address)
ME_latitude = ME_location.latitude
ME_longitude = ME_location.longitude

# determine the midpoint for centering the map
Mid_latitude = (ME_latitude + OR_latitude)/2
Mid_longitude = (ME_longitude + OR_longitude)/2

# make a map of the US with markers for each Portland
zoom=4
US_map = folium.Map(location=[Mid_latitude, Mid_longitude], zoom_start=zoom, no_touch=True, min_zoom=zoom, max_zoom=zoom,
                   zoom_control=False)
folium.Marker([OR_latitude, OR_longitude]).add_to(US_map)
folium.Marker([OR_latitude, OR_longitude], 
              icon=DivIcon(icon_size=(200,150),icon_anchor=(60,10),
                           html='<h4>Portland, Oregon</h4>')
             ).add_to(US_map)
folium.Marker([ME_latitude, ME_longitude]).add_to(US_map)
folium.Marker([ME_latitude, ME_longitude], 
              icon=DivIcon(icon_size=(200,150),icon_anchor=(60,10),
                           html='<h4>Portland, Maine</h4>')
             ).add_to(US_map)
US_map



## Data

Data sources for this project will include the United States Census Bureau and Foursquare.

### The United States Census Bureau

The Census Bureau provides a variety of demographic information collected through the American Community Survey. Selected data for the two Portlands is shown below.

In [3]:
# Census data for Portland, OR, and Portland, ME:
census_key='e53ba2a2cec0067578ae6768636a64b4c4f04e0b'

census_variables = {'S0101_C01_001E' : 'Total Population',
                    'S0101_C01_002E' : 'Under 5 yrs',
                    'S0101_C01_003E' : '5 to 9 yrs',
                    'S0101_C01_004E' : '10 to 14 yrs',
                    'S0101_C01_005E' : '15 to 19 yrs',
                    'S0101_C01_006E' : '20 to 24 yrs',
                    'S0101_C01_007E' : '25 to 29 yrs',
                    'S0101_C01_008E' : '30 to 34 yrs',
                    'S0101_C01_009E' : '35 to 39 yrs',
                    'S0101_C01_010E' : '40 to 44 yrs',
                    'S0101_C01_011E' : '45 to 49 yrs',
                    'S0101_C01_012E' : '50 to 54 yrs',
                    'S0101_C01_013E' : '55 to 59 yrs',
                    'S0101_C01_014E' : '60 to 64 yrs',
                    'S0101_C01_015E' : '65 to 69 yrs',
                    'S0101_C01_016E' : '70 to 74 yrs',
                    'S0101_C01_017E' : '75 to 79 yrs',
                    'S0101_C01_018E' : '80 to 84 yrs',
                    'S0101_C01_019E' : 'over 85 yrs',
                    'S0101_C03_001E' : 'Male population',
                    'S0101_C05_001E' : 'Female population',
                    'S0102_C01_006E' : 'Race: White',
                    'S0102_C01_007E' : 'Race: Black or African American',
                    'S0102_C01_008E' : 'Race: American Indian or Alaskan Native',
                    'S0102_C01_009E' : 'Race: Asian',
                    'S0102_C01_010E' : 'Race: Native Hawaiian or Other Pacific Islander',
                    'S0102_C01_011E' : 'Race: Other',
                    'S0102_C01_012E' : 'Race: Multiple',
                    'S0102_C01_013E' : 'Race: Hispanic or Latino - Any race',
                    'S0102_C01_037E' : 'Education: Bachelors degree or higher',
                    'S0102_C01_087E' : 'Population below poverty level'}

ME_place='60545'
ME_state='23'

OR_place='59000'
OR_state='41'

# load all data for 2019

ME_url='https://api.census.gov/data/2019/acs/acs5/subject?get=NAME,S0101_C01_001E,S0101_C01_002E,S0101_C01_003E,\
S0101_C01_004E,S0101_C01_005E,S0101_C01_006E,S0101_C01_007E,S0101_C01_008E,S0101_C01_009E,S0101_C01_010E,\
S0101_C01_011E,S0101_C01_012E,S0101_C01_013E,S0101_C01_014E,S0101_C01_015E,S0101_C01_016E,S0101_C01_017E,\
S0101_C01_018E,S0101_C01_019E,S0101_C03_001E,S0101_C05_001E,S0102_C01_006E,S0102_C01_007E,S0102_C01_008E,\
S0102_C01_009E,S0102_C01_010E,S0102_C01_011E,S0102_C01_012E,S0102_C01_013E,S0102_C01_037E,S0102_C01_087E\
&for=place:'+ME_place+'&in=state:'+ME_state+'&key='+census_key

ME_data=pd.read_json(ME_url)

OR_url='https://api.census.gov/data/2019/acs/acs5/subject?get=NAME,S0101_C01_001E,S0101_C01_002E,S0101_C01_003E,\
S0101_C01_004E,S0101_C01_005E,S0101_C01_006E,S0101_C01_007E,S0101_C01_008E,S0101_C01_009E,S0101_C01_010E,\
S0101_C01_011E,S0101_C01_012E,S0101_C01_013E,S0101_C01_014E,S0101_C01_015E,S0101_C01_016E,S0101_C01_017E,\
S0101_C01_018E,S0101_C01_019E,S0101_C03_001E,S0101_C05_001E,S0102_C01_006E,S0102_C01_007E,S0102_C01_008E,\
S0102_C01_009E,S0102_C01_010E,S0102_C01_011E,S0102_C01_012E,S0102_C01_013E,S0102_C01_037E,S0102_C01_087E\
&for=place:'+OR_place+'&in=state:'+OR_state+'&key='+census_key

OR_data=pd.read_json(OR_url)

# create dataframe for with data from both cities

Portland_data=ME_data.append(OR_data.iloc[1])


In [4]:
# load population data for 2010-2019
years=['2010','2011','2012','2013','2014','2015','2016','2017','2018','2019']

ME_population=[]
OR_population=[]

for year in years:
    ME_url='https://api.census.gov/data/'+year+'/acs/acs5/subject?get=S0101_C01_001E&for=place:'+ME_place+\
    '&in=state:'+ME_state+'&key='+census_key
    OR_url='https://api.census.gov/data/'+year+'/acs/acs5/subject?get=S0101_C01_001E&for=place:'+OR_place+\
    '&in=state:'+OR_state+'&key='+census_key
    
    ME_result=pd.read_json(ME_url)
    OR_result=pd.read_json(OR_url)
    
    ME_population.append(ME_result.iloc[1,0])
    OR_population.append(OR_result.iloc[1,0])

In [5]:
population_data=pd.DataFrame([years,ME_population,OR_population]).T
population_data.rename(columns={0:'Year',1:'Portland, Maine',2:'Portland, Oregon'},inplace=True)
population_data.set_index('Year',inplace=True)
population_data

,"Portland, Maine","Portland, Oregon"
Year,,
2010,66138,566686
2011,66240,576543
2012,66235,585888
2013,66227,594687
2014,66317,602568
2015,66490,612206
2016,66649,620589
2017,66715,630331
2018,66735,639387


In [6]:
# create dataframe for with data from both cities

Portland_data=ME_data.append(OR_data.iloc[1]).reset_index(drop=True)

# replace column names with variable descriptions and transpose
Portland_data.columns=Portland_data.iloc[0]
Portland_data.drop(index=0,inplace=True)
Portland_data=Portland_data.rename(columns=census_variables).T
Portland_data=Portland_data.rename(columns={1:'Portland, Maine',2:'Portland, Oregon'}).drop(index=['NAME'])
Portland_data.index.rename('',inplace=True)
Portland_data.drop(index=['place','state'],inplace=True)
Portland_data

,"Portland, Maine","Portland, Oregon"
,,
Total Population,66595,645291
Under 5 yrs,3218,34127
5 to 9 yrs,3066,33483
10 to 14 yrs,2600,30126
15 to 19 yrs,2704,30064
20 to 24 yrs,5292,38500
25 to 29 yrs,8140,63651
30 to 34 yrs,6282,64926
35 to 39 yrs,4701,61344


### Foursquare

Foursquare provides data on businesses and other venues in cities and towns. For example, Foursquare can be used to determine the five most common types of restaurant in a community, as shown below.

In [7]:
# Foursquare credentials
CLIENT_ID = 'CUGM1DGKBIM4FSUB4V20QNBMB1Y55IVBM2DWDABF1540BLCP' 
CLIENT_SECRET = 'U3RCUGOO4QJ5W5CMYAOAFRT34VNQ0T5KRJMQ4JSD0MKSSHXC' 
ACCESS_TOKEN = 'THNQFZIQ4IZC4GZNYD4SFFKMUPYMAL1TG1W21ZEFC4ZTYNPP' 
VERSION = '20180604'
LIMIT = 1000

# build query
search_query = 'Restaurant'
ME_radius = 7000
OR_radius = 11000
# a list of food categories
categories = ['503288ae91d4c4b30a586d67','4bf58dd8d48988d1c8941735','4bf58dd8d48988d14e941735','4bf58dd8d48988d142941735',\
              '56aa371be4b08b9a8d573568','52e81612bcbc57f1066b7a03','4bf58dd8d48988d145941735','4eb1bd1c3b7b55596b4a748f',\
              '52e81612bcbc57f1066b79fb','4deefc054765f83613cdba6f','4bf58dd8d48988d111941735','4bf58dd8d48988d113941735',\
              '4bf58dd8d48988d156941735','4eb1d5724b900d56c88a45fe','4bf58dd8d48988d1d1941735','56aa371be4b08b9a8d57350e',\
              '4bf58dd8d48988d149941735','52af39fb3cf9994f4e043be9','4bf58dd8d48988d14a941735','4bf58dd8d48988d169941735',\
              '52e81612bcbc57f1066b7a01','4bf58dd8d48988d1df931735','5e179ee74ae8e90006e9a746','52e81612bcbc57f1066b7a02',\
              '52e81612bcbc57f1066b79f1','4bf58dd8d48988d16c941735','4bf58dd8d48988d128941735','4bf58dd8d48988d16d941735',\
              '4bf58dd8d48988d17a941735','4bf58dd8d48988d144941735','4bf58dd8d48988d154941735','5293a7d53cf9994f4e043a45',\
              '52e81612bcbc57f1066b7a00','52e81612bcbc57f1066b79f2','52f2ae52bcbc57f1066b8b81','4bf58dd8d48988d147941735',\
              '4bf58dd8d48988d108941735','5744ccdfe4b0c0459246b4d0','4bf58dd8d48988d109941735','52e81612bcbc57f1066b7a05',\
              '4bf58dd8d48988d10b941735','4bf58dd8d48988d16e941735','4edd64a0c7ddd24ca188df1a','52e81612bcbc57f1066b7a09',\
              '4bf58dd8d48988d10c941735','4d4ae6fc7a7b7dea34424761','4bf58dd8d48988d155941735','4bf58dd8d48988d10d941735',\
              '4bf58dd8d48988d10e941735','52e81612bcbc57f1066b79ff','52e81612bcbc57f1066b79fe','4bf58dd8d48988d16f941735',\
              '52e81612bcbc57f1066b79fa','4bf58dd8d48988d10f941735','52e81612bcbc57f1066b7a06','4bf58dd8d48988d110941735',\
              '52e81612bcbc57f1066b79fd','5283c7b4e4b094cb91ec88d7','4bf58dd8d48988d1be941735','4bf58dd8d48988d1cd941735',\
              '4bf58dd8d48988d107941735','4bf58dd8d48988d16b941735','58daa1558bbb0b01f18ec1f4','4eb1bfa43b7b52c0e1adc2e8',\
              '56aa371be4b08b9a8d573558','4bf58dd8d48988d1bf941735','4bf58dd8d48988d1c0941735','4bf58dd8d48988d1c3941735',\
              '4bf58dd8d48988d1c1941735','4bf58dd8d48988d115941735','5bae9231bedf3950379f89e1','5bae9231bedf3950379f89e7',\
              '56aa371be4b08b9a8d573529','5744ccdfe4b0c0459246b4ca','58daa1558bbb0b01f18ec1cd','52e81612bcbc57f1066b79f7',\
              '5bae9231bedf3950379f89e4','5bae9231bedf3950379f89da','5bae9231bedf3950379f89ea','52e81612bcbc57f1066b79f9',\
              '4bf58dd8d48988d1c2941735','52e81612bcbc57f1066b79f8','4bf58dd8d48988d1c4941735','5293a7563cf9994f4e043a44',\
              '4bf58dd8d48988d1bd941735','4bf58dd8d48988d1c5941735','4bf58dd8d48988d1c6941735','5744ccdde4b0c0459246b4a3',\
              '4bf58dd8d48988d1ce941735','56aa371be4b08b9a8d57355a','4bf58dd8d48988d14f941735','4bf58dd8d48988d150941735',\
              '5413605de4b0ae91d18581a9','4bf58dd8d48988d1cc941735','4bf58dd8d48988d158941735','4bf58dd8d48988d1dc931735',\
              '56aa371be4b08b9a8d573538','4f04af1f2fb6e1c99f3db0bb','52e928d0bcbc57f1066b7e96','4bf58dd8d48988d1d3941735',\
              '4bf58dd8d48988d14c941735'] 

# start dataframes
ME_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}&categoryId={}'\
.format(CLIENT_ID,CLIENT_SECRET,ME_latitude,ME_longitude,VERSION,search_query,ME_radius,LIMIT,'4d4b7105d754a06374d81259')
OR_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}&categoryId={}'\
.format(CLIENT_ID,CLIENT_SECRET,OR_latitude,OR_longitude,VERSION,search_query,OR_radius,LIMIT,'4d4b7105d754a06374d81259')
    
# get the data
ME_result=requests.get(ME_url).json()
OR_result=requests.get(OR_url).json()
    
# normalize results
ME_restaurants=pd.DataFrame(json_normalize(ME_result['response']['venues']))
OR_restaurants=pd.DataFrame(json_normalize(OR_result['response']['venues']))


In [8]:
for category in categories:   
    ME_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}&categoryId={}'\
    .format(CLIENT_ID,CLIENT_SECRET,ME_latitude,ME_longitude,VERSION,search_query,ME_radius,LIMIT,category)
    OR_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}&categoryId={}'\
    .format(CLIENT_ID,CLIENT_SECRET,OR_latitude,OR_longitude,VERSION,search_query,OR_radius,LIMIT,category)
    
    # get the data
    ME_result=requests.get(ME_url).json()
    OR_result=requests.get(OR_url).json()
    
    # normalize results
    ME_group=pd.DataFrame(json_normalize(ME_result['response']['venues']))
    OR_group=pd.DataFrame(json_normalize(OR_result['response']['venues']))

    # add to dataframes
    ME_restaurants=ME_restaurants.append(ME_group)
    OR_restaurants=OR_restaurants.append(OR_group)  

In [9]:
# remove duplicates
ME_restaurants.drop_duplicates(subset=['id'],inplace=True)
OR_restaurants.drop_duplicates(subset=['id'],inplace=True)

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in ME_restaurants.columns if col.startswith('location.')] + ['id']
ME_restaurants = ME_restaurants.loc[:, filtered_columns]
OR_restaurants = OR_restaurants.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
ME_restaurants['categories'] = ME_restaurants.apply(get_category_type, axis=1)
OR_restaurants['categories'] = OR_restaurants.apply(get_category_type, axis=1)

# clean column names by keeping only last term
ME_restaurants.columns = [column.split('.')[-1] for column in ME_restaurants.columns]
OR_restaurants.columns = [column.split('.')[-1] for column in OR_restaurants.columns]


In [10]:
ME_top5_food=pd.DataFrame(ME_restaurants['categories'].value_counts().head(5)).reset_index()
OR_top5_food=pd.DataFrame(OR_restaurants['categories'].value_counts().head(5)).reset_index()

In [11]:
top5_food=pd.DataFrame([1,2,3,4,5],columns=['Rank'])
top5_food.insert(loc=1,column='Portland, Maine',value=ME_top5_food['index'])
top5_food.insert(loc=2,column='Portland, Oregon',value=OR_top5_food['index'])
top5_food.set_index('Rank')

,"Portland, Maine","Portland, Oregon"
Rank,,
1,American Restaurant,Chinese Restaurant
2,Italian Restaurant,Mexican Restaurant
3,Seafood Restaurant,American Restaurant
4,Thai Restaurant,Thai Restaurant
5,Mexican Restaurant,Sushi Restaurant
